In [1]:
import os, json, cv2, numpy as np
from matplotlib import pyplot as plt
from skimage import measure

In [5]:
class Annotator:
    def __init__(self, categories, super_category) -> None:
        self.super_category = super_category
        self.categories = categories
        self.increment_id = 0
        pass

    def create_annotation_format(self, contour, image_id, category_id):
        return {
            "iscrowd": 0,
            "id": self.increment_id,
            "image_id": image_id,
            "category_id": category_id,
            "bbox": cv2.boundingRect(contour),
            "area": cv2.contourArea(contour),
            "segmentation": [contour.flatten().tolist()],
        }

    def create_image_annotation(self, mask_image):
        gray = cv2.cvtColor(mask_image, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(
            gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        contours = cv2.findContours(
            thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[0]
        return contours

    def create_image_annotation_info(self, image_id, path, file_name, width, height):
        return {
            "file_name": file_name,
            "height": height,
            "id": image_id,
            "path": path,
            "width": width,
        }

    def _runImageAnnotation(self, mask_path):
        annotation_count = 0
        image_id = 0
        annotations = []
        images = []

        for subdir, dirs, files in os.walk(mask_path):
            for file in files:
                category = subdir.split('/')[-1]
                category_id = self.categories[category]
                image_path = os.path.join(subdir, file)
                mask_image = cv2.imread(image_path)
                height, width, _ = mask_image.shape

                image = self.create_image_annotation_info(image_id,
                                                          image_path, file, width, height)
                images.append(image)
                contours = self.create_image_annotation(
                    mask_image)

                for contour in contours:
                    annotation = self.create_annotation_format(
                        contour, image_id, category_id)
                    annotations.append(annotation)
                    self.increment_id += 1
                    annotation_count += 1
                image_id += 1
        return images, annotations, annotation_count

    def create_category_annotation(self):
        category_list = []
        for key, value in self.categories.items():
            category = {"id": value, "name": key, "supercategory": self.super_category}
            category_list.append(category)
        return category_list

    def create_coco_json_format(self):
        return {
            "annotations": [{}],
            "categories": [{}],
            "images": [{}],
            "info": {},
            "licenses": [],
        }

    def fromMaskImage(self, dir_list):
        coco_format = self.create_coco_json_format()

        for dir in dir_list:
            parent_dir_name = dir.split('/')[0]
            dir_name = dir.split('/')[1]
            mask_path = f"dataset/{dir}/"
            # Create category info
            coco_format['categories'] = self.create_category_annotation()
            # Create images and annotation
            coco_format['images'], coco_format['annotations'], annotation_count = self._runImageAnnotation(
                mask_path)

            with open(f"dataset/{parent_dir_name}/{dir_name}.json", "w") as outfile:
                json.dump(coco_format, outfile, sort_keys=True, indent=2)

            print(
                f"Created {annotation_count} annotations for images in folder: {mask_path}")


In [6]:
annotator = Annotator(categories={'normal': 0, 'covid-19': 1, 'non-covid': 2}, super_category='lung')

In [8]:
fold1 = annotator.fromMaskImage(['Fold1/train_mask', 'Fold1/test_mask', 'Fold1/validation_mask'])
fold2 = annotator.fromMaskImage(['Fold2/train_mask', 'Fold2/test_mask', 'Fold2/validation_mask'])
fold3 = annotator.fromMaskImage(['Fold3/train_mask', 'Fold3/test_mask', 'Fold3/validation_mask'])
fold4 = annotator.fromMaskImage(['Fold4/train_mask', 'Fold4/test_mask', 'Fold4/validation_mask'])
fold5 = annotator.fromMaskImage(['Fold5/train_mask', 'Fold5/test_mask', 'Fold5/validation_mask'])

Created 2160 annotations for images in folder: dataset/Fold1/train_mask/
Created 300 annotations for images in folder: dataset/Fold1/test_mask/
Created 540 annotations for images in folder: dataset/Fold1/validation_mask/
Created 2160 annotations for images in folder: dataset/Fold2/train_mask/
Created 300 annotations for images in folder: dataset/Fold2/test_mask/
Created 540 annotations for images in folder: dataset/Fold2/validation_mask/
Created 2160 annotations for images in folder: dataset/Fold3/train_mask/
Created 300 annotations for images in folder: dataset/Fold3/test_mask/
Created 540 annotations for images in folder: dataset/Fold3/validation_mask/
Created 2160 annotations for images in folder: dataset/Fold4/train_mask/
Created 300 annotations for images in folder: dataset/Fold4/test_mask/
Created 540 annotations for images in folder: dataset/Fold4/validation_mask/
Created 2160 annotations for images in folder: dataset/Fold5/train_mask/
Created 300 annotations for images in folde